# Objetivo

Aqui estão as funções responsaveis por filtrar os dados fornecidos pelo facebook para algo mais facil de se usar
Ou criar mensagens 

In [1]:
# Necessário para o envio de mensagens
import json
import requests
# Necessário para o recebimento de mensagens 
from flask import Blueprint, request, jsonify, current_app 
import hashlib
import hmac

# Classe Responsavel por filtar informações recebidas da mensagem

In [ ]:
class WhatsData:
    '''
Pega informações cruciais na mensagem:
name
type
wa_id
text
    '''
    def __init__(self, data):
        self.data = data
        self.process_data()

    # essa linha esta mal implementada,
    # se o dono do número não tiver o 9 no numero, ele adere.
    # contudo, so funciona pra numeros do brasil...
    def split(self, string):
        string = str(string)
        if len(string) == 12:
            self.string = string[:4] + '9' + string[4:]
        else:
            self.string = string

    def check_nested_path(self, path):
        """Checa se um caminho existe."""
        current_level = self.data
        # Informações do whatsapp vêm bagunçadas, mistura de lista com dicionario...
        for key in path:
            if isinstance(current_level, dict):
                if key in current_level:
                    current_level = current_level[key]
                else:
                    return False
            elif isinstance(current_level, list):
                try:
                    index = int(key)
                    if 0 <= index < len(current_level):
                        current_level = current_level[index]
                    else:
                        return False
                except ValueError:
                    return False
            else:
                return False

        return True


    def go_to_nested_path(self, path):
        """Vai para o path se ele existe."""
        current_level = self.data

        for key in path:
            if isinstance(current_level, dict):
                if key in current_level:
                    current_level = current_level[key]
                else:
                    return None
            elif isinstance(current_level, list):
                try:
                    index = int(key)
                    if 0 <= index < len(current_level):
                        current_level = current_level[index]
                    else:
                        return None
                except ValueError:
                    return None
            else:
                return None

        return current_level
    
    def process_data(self):
        """Processa o tipo de informação."""
        path = {
            'is_read'        : ['entry', '0', 'changes', '0', 'value', 'statuses'],
            'is_message'     : ['entry', '0', 'changes', '0', 'value', 'contacts'],
            'read_wa_id'     : ['entry', '0', 'changes', '0', 'value', 'statuses', '0', 'recipient_id'],
            'messanger_wa_id': ['entry', '0', 'changes', '0', 'value', 'contacts', '0', 'wa_id'],
            'messanger_name' : ['entry', '0', 'changes', '0', 'value', 'contacts', '0', 'profile', 'name'],
            'message_type'   : ['entry', '0', 'changes', '0', 'value', 'messages', '0', 'type'],
            'message_text'   : ['entry', '0', 'changes', '0', 'value', 'messages', '0', 'text', 'body']
        }
        # Verifica se a mensagem é confirmação de lida
        if self.check_nested_path(path['is_read']):
            self.split(self.go_to_nested_path(path['read_wa_id']))
            self.wa_id = self.string
            self.name = None
            self.type = 'read'
            self.text = None
        # verifica se é uma mensagem
        if self.check_nested_path(path['is_message']):
            self.split(self.go_to_nested_path(path['messanger_wa_id']))
            self.wa_id = self.string
            self.name = self.go_to_nested_path(path['messanger_name'])
            self.type = self.go_to_nested_path(path['message_type'])
            if self.type == 'text':
                self.text = self.go_to_nested_path(path['message_text'])
            else:
                self.text = None



# Manda mensagem templete

Mensagem precisa ser aprovada pelo facebook, e é a unica mensagem que o bot pode mandar para um usuario se ele não responder

In [ ]:
def send_whatsapp_message(RECIPIENT_WAID, app):

    url = f"https://graph.facebook.com/{app.config['VERSION']}/{app.config['PHONE_NUMBER_ID']}/messages"
    headers = {
        "Authorization": "Bearer " + app.config['ACCESS_TOKEN'],
        "Content-Type": "application/json",
    }
    data = {
        "messaging_product": "whatsapp",
        "to": RECIPIENT_WAID,
        "type": "template",
        "template": {"name": "hello_world", "language": {"code": "en_US"}},
    }
    response = requests.post(url, headers=headers, json=data)
    return response

# Cria templete para que o whats app possa mandar a mensagem

In [ ]:
def get_text_message_input(RECIPIENT_WAID, text):
    return json.dumps(
        {
            "messaging_product": "whatsapp",
            "recipient_type": "individual",
            "to": RECIPIENT_WAID,
            "type": "text",
            "text": {"preview_url": False, "body": text},
        }
    )

# Envia mensagem

Quando formatada

In [ ]:
def send_message(data, app):
    headers = {
        "Content-type": "application/json",
        "Authorization": f"Bearer {app.config['ACCESS_TOKEN']}",
    }

    url = f"https://graph.facebook.com/{app.config['VERSION']}/{app.config['PHONE_NUMBER_ID']}/messages"

    response = requests.post(url, data=data, headers=headers)
    if response.status_code == 200:
        print("Status:", response.status_code)
        print("Content-type:", response.headers["content-type"])
        print("Body:", response.text)
        return response
    else:
        print(response.status_code)
        print(response.text)
        return response
    

# Recebe mensagem

## Cria instancia do Flask

In [ ]:
webhook_blueprint = Blueprint("webhook", __name__)

## Metodo de verificar assinatura

In [ ]:
def verify_signature(request, app):
    received_signature = request.headers.get('x-hub-signature-256')
    if not received_signature:
        return False
    
    # Generate the expected signature
    payload = request.get_data()
    expected_signature = 'sha256=' + hmac.new(
        key=app.config['APP_SECRET'].encode(),
        msg=payload,
        digestmod=hashlib.sha256
    ).hexdigest()
    
    # Compare the signatures
    return hmac.compare_digest(received_signature, expected_signature)

## Recebe a mensagem e executa a função

In [ ]:
@webhook_blueprint.route('/webhook', methods=['GET', 'POST'])
def webhook():
    if request.method == 'GET':
        # Facebook webhook verification
        mode = request.args.get('hub.mode')
        token = request.args.get('hub.verify_token')
        challenge = request.args.get('hub.challenge')

        if mode and token:
            if mode == 'subscribe' and token == current_app.config['VERIFICATION_TOKEN']:
                print("Webhook verified")
                return challenge, 200
            else:
                return 'Verification token mismatch', 403

    if request.method == 'POST':
        # Verify the request signature
        if not verify_signature(request, current_app):
            print("Invalid signature")
            return jsonify({'status': 'error', 'message': 'Invalid signature'}), 403

        # Handle the incoming webhook event
        # Essa parte que devemos alterar 
        data = request.json # recebe a mensagem em formato de data
        do_it(data, current_app)
        #print(f"Received webhook: {data}")
        return jsonify({'status': 'success'}), 200